Modelo como proposto por Sambasivan & Yahya (2005)

In [173]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import numpy as np

# Entrada de dados

In [174]:
def read_dat_file(file_path):
    """"Função para a leitura das instâncias de Mariá"""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # 1. Lendo quantidade de itens e períodos
    items, periods = map(int, lines[0].split())

    # 2. Lendo número de plantas
    num_plants = int(lines[1].strip())

    # 3. Lendo capacidades das plantas
    capacities = [int(lines[i + 2].strip()) for i in range(num_plants)]
    capacities = np.tile(capacities, (periods, 1)).T  # Repete as capacidades ao longo dos períodos (deixar na forma j, t)

    # 4. Lendo a matriz de produção (tempo de produção, tempo de setup, custo de setup, custo de produção)
    production_data = []
    start_line = 2 + num_plants
    production_time = np.zeros((items, num_plants))  # Inicializar listas para armazenar separadamente os tempos e custos
    setup_time = np.zeros((items, num_plants))
    setup_cost = np.zeros((items, num_plants))
    production_cost = np.zeros((items, num_plants))
    for i in range(num_plants * items):  # Preencher as matrizes com os dados lidos
        plant = i // items  # Determina a planta
        item = i % items    # Determina o item
        # Extrair os dados de cada linha
        prod_time, set_time, set_cost, prod_cost = map(float, lines[start_line + i].split())
        production_time[item, plant] = prod_time  # Preencher as respectivas matrizes
        setup_time[item, plant] = set_time
        setup_cost[item, plant] = set_cost
        production_cost[item, plant] = prod_cost

    # 5. Lendo os custos de inventário
    inventory_costs_line = start_line + num_plants * items
    inventory_costs = list(map(float, lines[inventory_costs_line].split()))  # Lê todos os valores de inventory_costs como uma única lista
    inventory_costs = np.array(inventory_costs).reshape(num_plants, -1)  # Divide a lista de custos de inventário por planta
    inventory_costs = inventory_costs.T  # Deixa na forma (i, j)

    # 6. Lendo a matriz de demanda (12 linhas, 12 colunas)
    demand_matrix = []
    demand_start_line = inventory_costs_line + 1

    # Verifica se a matriz de demanda está na forma padrão
    num_demand_lines = 0
    for i in range(demand_start_line, len(lines)):
        if lines[i].strip():  # Se a linha não estiver vazia, contamos como linha de demanda
            num_demand_lines += 1
        else:
            break  # Interrompe se encontrar uma linha vazia (ou um bloco separado)
    
    # Calcula o multiplicador, caso tenha mais linhas do que períodos
    if num_demand_lines == periods * 2:
        multiplier = 2  # A matriz de demanda tem o dobro de linhas
    else:
        multiplier = 1  # A matriz de demanda tem o número esperado de linhas (T)
    
    # Leitura inicial das demandas
    for i in range(periods * multiplier):  # Lê as linhas de demandas para os períodos
        demands = list(map(int, lines[demand_start_line + i].split()))
        demand_matrix.append(demands)

    # print('Linha 1:', demand_matrix[0])
    # print('Linha 13:', demand_matrix[periods], '\n\n')
    
    # Agora, se multiplier == 2, precisamos combinar as linhas 1-12 com 13-24 corretamente
    if multiplier == 2:
        new_demand_matrix = []
        
        for i in range(periods):  # Para cada linha do primeiro bloco (1-12)
            combined_period_demand = []  # Lista única para armazenar a demanda combinada para o período i
            
            # Quebrar a linha i entre as plantas
            demands_first_part = demand_matrix[i]
            demands_second_part = demand_matrix[i + periods]  # Linha correspondente do segundo bloco
            
            # Calcular o número de elementos por planta
            elements_per_plant_first = len(demands_first_part) // num_plants
            elements_per_plant_second = len(demands_second_part) // num_plants
            
            # Concatenar as demandas para cada planta
            for j in range(num_plants):
                # Particionar a demanda de cada planta
                plant_demand_first = demands_first_part[j*elements_per_plant_first:(j+1)*elements_per_plant_first]
                plant_demand_second = demands_second_part[j*elements_per_plant_second:(j+1)*elements_per_plant_second]
                
                # Concatenar as demandas da planta j (primeiro e segundo blocos) e adicionar diretamente à lista única
                combined_period_demand.extend(plant_demand_first + plant_demand_second)
            
            # Adicionar a demanda ajustada (em formato de lista única) do período à nova matriz de demanda
            new_demand_matrix.append(combined_period_demand)
        
        # Substitui a matriz de demandas pela combinada e particionada por planta
        demand_matrix = new_demand_matrix
    
    # print('Linha 1 output após junção:', demand_matrix[0])

    # Agora vamos dividir os valores de cada linha combinada entre as plantas
    final_demand_matrix = []
    for demands in demand_matrix:
        period_demand = []
        for j in range(num_plants):
            # Divide a demanda combinada por planta, assumindo que cada planta tem o mesmo número de itens
            plant_demand = demands[j*items:(j+1)*items]
            period_demand.append(plant_demand)
        final_demand_matrix.append(period_demand)
    
    # Transpor a matriz de demanda para o formato correto (itens, plantas, períodos)
    final_demand_matrix = np.array(final_demand_matrix)
    # print(final_demand_matrix)
    final_demand_matrix = np.transpose(final_demand_matrix, (2, 1, 0))  # Converte para o formato (itens, plantas, períodos)

    # 7. Lendo os custos de transferência
    transfer_costs = []
    transfer_cost_line = demand_start_line + periods * multiplier
    while transfer_cost_line < len(lines):
        line = lines[transfer_cost_line].strip()  # Verificar se a linha não está vazia antes de tentar ler
        if line:
            transfer_costs.append(float(line))
        transfer_cost_line += 1

    def create_transfer_cost_matrix(transfer_costs, num_plants):  # Criar a matriz de custos de transferência (simétrica)
        transfer_cost_matrix = np.zeros((num_plants, num_plants))  # Inicializar a matriz de zeros
        if len(transfer_costs) == 1:
            transfer_cost = transfer_costs[0]  # Se houver apenas um custo de transferência, aplicar para todos os pares de plantas
            for j in range(num_plants):
                for k in range(j + 1, num_plants):
                    transfer_cost_matrix[j, k] = transfer_cost
                    transfer_cost_matrix[k, j] = transfer_cost
        else:
            idx = 0  # Se houver múltiplos custos, aplicar entre pares de plantas
            for j in range(num_plants):
                for k in range(j + 1, num_plants):
                    transfer_cost_matrix[j, k] = transfer_costs[idx]
                    transfer_cost_matrix[k, j] = transfer_costs[idx]
                    idx += 1
        return transfer_cost_matrix

    transfer_costs = create_transfer_cost_matrix(transfer_costs, num_plants)

    return {"items": items,
            "periods": periods,
            "num_plants": num_plants,
            "capacities": capacities,
            "production_time": production_time,
            "setup_time": setup_time,
            "setup_cost": setup_cost,  
            "production_cost": production_cost,
            "inventory_costs": inventory_costs,
            "demand_matrix": final_demand_matrix,
            "transfer_costs": transfer_costs}

In [175]:
# Exemplo de uso
file_path = '../instancias/maria_desiree/ABA012450_1.dat'
data = read_dat_file(file_path)
display(data)

{'items': 50,
 'periods': 12,
 'num_plants': 4,
 'capacities': array([[13274, 13274, 13274, 13274, 13274, 13274, 13274, 13274, 13274,
         13274, 13274, 13274],
        [14036, 14036, 14036, 14036, 14036, 14036, 14036, 14036, 14036,
         14036, 14036, 14036],
        [14401, 14401, 14401, 14401, 14401, 14401, 14401, 14401, 14401,
         14401, 14401, 14401],
        [13632, 13632, 13632, 13632, 13632, 13632, 13632, 13632, 13632,
         13632, 13632, 13632]]),
 'production_time': array([[1.9, 4.3, 4.4, 4. ],
        [3. , 4.8, 1.2, 1.1],
        [3.4, 1.4, 3.2, 3.2],
        [1.7, 4.3, 4.7, 1.4],
        [4.4, 3.8, 4.5, 3. ],
        [2.3, 1.5, 1.4, 3.5],
        [2. , 3.4, 2.4, 2.2],
        [2.1, 1.7, 1.3, 2.2],
        [2.9, 1.2, 3.2, 2.6],
        [2. , 4.2, 1.4, 4.7],
        [1.1, 2.1, 3.5, 1.6],
        [1.6, 2.7, 3. , 3.9],
        [4.1, 2. , 2.2, 2.7],
        [2.9, 5. , 1.5, 3.8],
        [3.7, 2.9, 1.7, 3.1],
        [4. , 3.4, 4.8, 3.3],
        [3.9, 1.8, 4.3, 1

In [176]:
# Produtos (i)
I = np.array([_ for _ in range(data['items'])])
# Plantas (j)
J = np.array([_ for _ in range(data['num_plants'])])
# Períodos (t)
T = np.array([_ for _ in range(data['periods'])])

In [177]:
Produtos = np.array([_ for _ in range(data['items'])])  # I
Plantas = np.array([_ for _ in range(data['num_plants'])])  # J
Periodos = np.array([_ for _ in range(data['periods'])])  # T

In [178]:
# Leitura de np.arrays
demanda = np.array(data['demand_matrix'])
capacidade_planta = np.array(data['capacities'])
tempo_producao = np.array(data['production_time'])
tempo_setup = np.array(data['setup_time'])
custo_producao = np.array(data['production_cost'])
custo_setup = np.array(data['setup_cost'])
custo_transporte = np.array(data['transfer_costs'])
custo_estoque = np.array(data['inventory_costs'])

In [179]:
# Conversão para dicts
demanda = {(int(i), int(j), int(t)): float(demanda[i, j, t]) for i in range(len(Produtos)) for j in range(len(Plantas)) for t in range(len(Periodos))}
capacidade_planta = {(int(j), int(t)): float(capacidade_planta[j, t]) for j in range(len(Plantas)) for t in range(len(Periodos))}
tempo_producao = {(int(i), int(j)): float(tempo_producao[i, j]) for i in range(len(Produtos)) for j in range(len(Plantas))}
tempo_setup = {(int(i), int(j)): float(tempo_setup[i, j]) for i in range(len(Produtos)) for j in range(len(Plantas))}
custo_producao = {(int(i), int(j)): float(custo_producao[i, j]) for i in range(len(Produtos)) for j in range(len(Plantas))}
custo_setup = {(int(i), int(j)): float(custo_setup[i, j]) for i in range(len(Produtos)) for j in range(len(Plantas))}
custo_transporte = {(int(i), int(j)): float(custo_transporte[i, j]) for i in range(len(Plantas)) for j in range(len(Plantas))}
custo_estoque = {(int(i), int(j)): float(custo_estoque[i, j]) for i in range(len(Produtos)) for j in range(len(Plantas))}

# Modelagem

In [180]:
model = pyo.ConcreteModel()

## Conjuntos

In [181]:
# Produtos
model.I = pyo.Set(initialize=Produtos)  # Índice i relativo aos produtos
# Plantas
model.J = pyo.Set(initialize=Plantas)  # Índice j relativo às plantas
# Períodos
model.T = pyo.Set(initialize=Periodos)  # Índice t relativo aos periodos

I, J, T = model.I, model.J, model.T

## Parâmetros

In [182]:
# Demanda
model.d = pyo.Param(I * J * T, initialize=demanda, within=pyo.NonNegativeReals)
# Capacidade
model.cap = pyo.Param(J * T, initialize=capacidade_planta, within=pyo.NonNegativeReals)
# Tempo de produção
model.b = pyo.Param(I * J, initialize=tempo_producao, within=pyo.NonNegativeReals)
# Tempo de setup
model.f = pyo.Param(I * J, initialize=tempo_setup, within=pyo.NonNegativeReals)
# Custo de produção
model.c = pyo.Param(I * J, initialize=custo_producao, within=pyo.NonNegativeReals)
# Custo de setup
model.s = pyo.Param(I * J, initialize=custo_setup, within=pyo.NonNegativeReals)
# Custo de transporte
model.r = pyo.Param(J * J, initialize=custo_transporte, within=pyo.NonNegativeReals)
# Custo de estoque
model.h = pyo.Param(I * J, initialize=custo_estoque, within=pyo.NonNegativeReals)

d, cap, b, f, c, s, r, h = model.d, model.cap, model.b, model.f, model.c, model.s, model.r, model.h

## Variáveis de decisão

In [183]:
# Quantidade produzida
model.X = pyo.Var(I * J * T, within=pyo.NonNegativeIntegers)
# Quantidade estocada
model.Q = pyo.Var(I * J * T, within=pyo.NonNegativeIntegers)
# Quantidade transportada
model.W = pyo.Var(I * J * J * T, within=pyo.NonNegativeIntegers)
# Variável de setup (binária)
model.Z = pyo.Var(I * J * T, within=pyo.Binary)

X, Q, W, Z = model.X, model.Q, model.W, model.Z

## Função objetivo

In [184]:
expr_objetivo = sum(sum(sum(c[i, j] * X[i, j, t] + h[i, j] * Q[i, j, t] + s[i, j] * Z[i, j, t] +
                            sum(r[j, k] * W[i, j, k, t] for k in J if k != j) for t in T) for j in J) for i in I) 
model.obj = pyo.Objective(sense=pyo.minimize,
                          expr=expr_objetivo)

objetivo = model.obj

## Restrições

In [185]:
# Balanço de estoque
model.r_balanco = pyo.ConstraintList()
for i in I:
    for j in J:
        for t in T:
            if t == 0:
                restr = Q[i, j, t] == X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t]
            else:
                restr = Q[i, j, t] == Q[i, j, t-1] + X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t]
            model.r_balanco.add(expr=restr)

In [186]:
# Restrição que obriga setup
model.r_setup = pyo.ConstraintList()
for i in I:
    for j in J:
        for t in T:
            restr = X[i, j, t] <= sum(sum(d[i, j, b] for b in range(t, len(T))) for j in J) * Z[i, j, t]
            model.r_setup.add(expr=restr)

In [187]:
# Restrição de capacidade
model.r_capacidade = pyo.ConstraintList()
for j in J:
    for t in T:
        restr = sum(b[i, j] * X[i, j, t] + f[i, j] * Z[i, j, t] for i in I) <= cap[j, t]
        model.r_capacidade.add(expr=restr)

# Solve Gurobi

In [ ]:
solver = SolverFactory("gurobi")
solver.options["TimeLimit"] = 1800
resultado = solver.solve(model, tee=False)

Interessante que o .lp fornecido pelo pyomo tem menos variáveis que o do Gurobipy (1440 contínuas contra 1728). Isso parece ser devido ao drop de variáveis W[i, j, k, t] com k == j (288 valores) que não são utilizadas.

In [ ]:
# Contagem de casos em que k == j
c = 0
for i in I:
    for j in J:
        for k in J:
            for t in T:
                if k == j:
                    c += 1
c

In [ ]:
resultado['Problem'][0]['Upper bound']

In [ ]:
resultado['Problem'][0]['Lower bound']

In [ ]:
resultado

# Solve CPLEX

In [ ]:
solver = SolverFactory("cplex")
solver.options["TimeLimit"] = 1800
resultado = solver.solve(model, tee=False)

In [ ]:
resultado['Problem'][0]['Upper bound']

In [ ]:
resultado['Problem'][0]['Lower bound']

In [ ]:
resultado